<a href="https://colab.research.google.com/github/hubertwel/paragraph-similarity/blob/main/paragraph-similarity/paragraph_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

sentences = ["I ate dinner.", 
       "We had a three-course meal.", 
       "Brad came to dinner with us.",
       "He loves fish tacos.",
       "In the end, we all felt like we ate too much.",
       "We all agreed; it was a magnificent evening."]

# Tokenization of each document
sentences_tk = []
tokenizer = RegexpTokenizer(r'\w+')
for s in sentences:
    sentence_tk = tokenizer.tokenize(s.lower())
    sentences_tk.append(sentence_tk)
print('tokenized sentences: ', sentences_tk)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
tokenized sentences:  [['i', 'ate', 'dinner'], ['we', 'had', 'a', 'three', 'course', 'meal'], ['brad', 'came', 'to', 'dinner', 'with', 'us'], ['he', 'loves', 'fish', 'tacos'], ['in', 'the', 'end', 'we', 'all', 'felt', 'like', 'we', 'ate', 'too', 'much'], ['we', 'all', 'agreed', 'it', 'was', 'a', 'magnificent', 'evening']]


In [6]:
pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 163kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
